In [1]:
import sklearn 
from sklearn.linear_model import LogisticRegression
import pandas as pd 
import numpy as np
from statsmodels.tsa.arima_model import ARMA
import datetime


/home/cody/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Load CSVs

In [2]:
regular_season_results = pd.read_csv("../ncaa_data/RegularSeasonDetailedResults.csv")
tourney_results = pd.read_csv("../ncaa_data/NCAATourneyDetailedResults.csv")
results = pd.concat([regular_season_results, tourney_results])

In [3]:
seasons = pd.read_csv("../ncaa_data/Seasons.csv")
seasons = seasons[["Season","DayZero"]]
results = results.set_index("Season").join(seasons.set_index("Season"), how="inner", rsuffix="season").reset_index()

# Build Team/Season Features

In [4]:
winning_results = results[['Season', 'DayNum','DayZero', 'WTeamID', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
losing_results = results[['Season', 'DayNum','DayZero', 'LTeamID', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]
winning_results.columns = ['Season', 'DayNum','DayZero', 'TeamID', 'Score','FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
'Ast', 'TO', 'Stl', 'Blk', 'PF']
losing_results.columns = ['Season', 'DayNum','DayZero', 'TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
'Ast', 'TO', 'Stl', 'Blk', 'PF']
team_df = pd.concat([winning_results, losing_results]).sort_values(["Season","DayNum"]).set_index("DayNum").groupby(["Season","TeamID"]).apply(lambda x: x.reindex(range(0,150), method='ffill'))


# Bring in Massey Ordinals 

In [5]:
massey = pd.read_csv("../ncaa_data/MasseyOrdinals.csv")
massey = massey.rename(columns={"RankingDayNum":"DayNum"})
massey = massey.pivot_table(index=["Season","TeamID","DayNum"], columns="SystemName", values="OrdinalRank")
massey = massey.fillna(method="ffill")
massey = massey.drop("DC2", axis=1)

In [6]:
team_df = team_df.join(massey, how="left")


# Clean Up and Make Some Columns MA

In [7]:
team_df = team_df.drop(["TeamID","Season"], axis=1)
deltas = team_df.reset_index()["DayNum"].apply(lambda x: datetime.timedelta(days=x))
team_df["date"] = pd.to_datetime(team_df["DayZero"]) + deltas.values

In [8]:
exclude = ["date","DayZero","Score","DayNum","Season"]
ma_columns = team_df.columns.tolist()
ma_columns = [item for item in ma_columns if item not in exclude]

In [9]:
ma_data = team_df.groupby(["Season","TeamID"])[ma_columns].apply(lambda x: x.rolling(window=10, min_periods=1).mean())

In [10]:
team_df[ma_columns] = ma_data

# Pickle Team DF

In [11]:
team_df.to_pickle("../ncaa_data/team_df.p")

# Build Game DF

In [12]:
games = results[["WTeamID","LTeamID","Season","DayNum","WLoc","DayZero","WScore","LScore"]]

In [13]:
games["Team1"] = games[["WTeamID","LTeamID"]].min(axis=1)
games["Team2"] = games[["WTeamID","LTeamID"]].max(axis=1)


/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [14]:
games["Team1Score"] = 0
games["Team2Score"] = 0
games["Team1"] = games[["WTeamID","LTeamID"]].min(axis=1)
games["Team2"] = games[["WTeamID","LTeamID"]].max(axis=1)
games.loc[games["WTeamID"] == games["Team1"],"Team1Score"] = games["WScore"]
games.loc[games["WTeamID"] != games["Team1"],"Team1Score"] = games[games["WTeamID"] != games["Team1"]]["LScore"]
games.loc[games["WTeamID"] == games["Team2"],"Team2Score"] = games["WScore"]
games.loc[games["WTeamID"] != games["Team2"],"Team2Score"] = games["LScore"]
# games["ScoreDiff"] = 
# games["Team1Home"] = (games["WLoc"]=="H") & (games["WTeamID"] == games["Team1"])
# games["Team2Home"] = (games["WLoc"]=="H") & (games["WTeamID"] == games["Team2"])

/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [15]:
games = games.drop(["WTeamID","LTeamID","WLoc","DayZero"], axis=1)


# Increment the dayNum so that we only use results up through the day previous to the game

In [16]:
incremented = team_df.reset_index()
incremented["DayNum"] = incremented["DayNum"] + 1
incremented = incremented.set_index(["Season","TeamID","DayNum"])
full_df = games.join(incremented, on=["Season", "Team1","DayNum"], how="right").join(incremented, on=["Season", "Team2","DayNum"], rsuffix="2", how="inner")

In [31]:
games.shape

(77617, 8)

# Add Some Diff Columns

In [17]:
for column in ma_columns:
    try:
        full_df[column + "diff"] = full_df[column] - full_df[str(column +"2")]
    except Exception as e:
        print(e)

In [18]:
full_df.to_pickle("../ncaa_data/full_df.p")

# Match Ups to Predict

In [19]:
pairs_to_predict = pd.read_csv("../ncaa_data/SampleSubmissionStage1.csv")["ID"].str.split("_")
seasons = pairs_to_predict.apply(lambda x: int(x[0]))
team1 = pairs_to_predict.apply(lambda x: int(x[1]))
team2 = pairs_to_predict.apply(lambda x:int(x[2]))

In [20]:
tourney_matchups = pd.DataFrame()
tourney_matchups["team1"] = team1
tourney_matchups["team2"] = team2
tourney_matchups["season"] = seasons

In [28]:
full_df.shape

(77570, 527)

In [21]:
tourney_matchups

,team1,team2,season
0,1107,1110,2014
1,1107,1112,2014
2,1107,1113,2014
3,1107,1124,2014
4,1107,1140,2014
5,1107,1142,2014
6,1107,1153,2014
7,1107,1157,2014
8,1107,1160,2014
9,1107,1163,2014


In [557]:
csv_file.close()

In [49]:
full_df[full_df["DayNum"]==145]

,Season,DayNum,WScore,LScore,Team1,Team2,Team1Score,Team2Score,DayZero,Score,...,UPSdiff,USAdiff,WILdiff,WLKdiff,WMRdiff,WOBdiff,WOLdiff,WTEdiff,YAGdiff,ZAMdiff
4673,2003,145,78,75,1112,1242,75,78,11/4/2002,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4674,2003,145,83,69,1246,1266,69,83,11/4/2002,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9308,2004,145,87,71,1104,1163,71,87,11/3/2003,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9309,2004,145,64,62,1329,1386,64,62,11/3/2003,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14047,2005,145,90,89,1112,1228,89,90,11/1/2004,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14048,2005,145,93,85,1257,1452,93,85,11/1/2004,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18868,2006,145,70,60,1261,1400,70,60,10/31/2005,62.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18869,2006,145,50,45,1272,1417,45,50,10/31/2005,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23975,2007,145,92,76,1272,1326,76,92,10/30/2006,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23976,2007,145,68,55,1242,1417,55,68,10/30/2006,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
